In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.5.1`
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

val spark = SparkSession.builder()
  .appName("Ques14")
  .master("local[*]")  // Use all cores
  .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/25 09:39:46 INFO SparkContext: Running Spark version 3.5.1
25/07/25 09:39:46 INFO SparkContext: OS info Windows 11, 10.0, amd64
25/07/25 09:39:46 INFO SparkContext: Java version 11.0.26
25/07/25 09:39:47 WARN Shell: Did not find winutils.exe: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
25/07/25 09:39:47 INFO ResourceUtils: ==============================================================
25/07/25 09:39:47 INFO ResourceUtils: No custom resources configured for spark.driver.
25/07/25 09:39:47 INFO ResourceUtils: ==============================================================
25/07/25 09:39:47 INFO SparkContext: Submitted application: Ques14
25/07/25 09:39:47 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory 

import $ivy.$                                      

import org.apache.spark.sql.SparkSession

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@3fa3c2cc

In [2]:
// Create a DataFrame from a sequence of data
import spark.implicits._

val schema = StructType(Seq(
    StructField("locality", IntegerType, true),
    StructField("type_of_property", StringType, true),
    StructField("subtype_of_property", StringType, true),
    StructField("price", IntegerType, true),
    StructField("no_of_rooms", IntegerType, true),
    StructField("open_fire", IntegerType, true),
    StructField("terrace", IntegerType, true),
    StructField("garden", IntegerType, true),
    StructField("swimming_pool", IntegerType, true),
    StructField("state_of_building", StringType, true),
    StructField("construction_year", IntegerType, true)
))

import spark.implicits._


schema: StructType = Seq(
  StructField(
    name = "locality",
    dataType = IntegerType,
    nullable = true,
    metadata = {}
  ),
  StructField(
    name = "type_of_property",
    dataType = StringType,
    nullable = true,
    metadata = {}
  ),
  StructField(
    name = "subtype_of_property",
    dataType = StringType,
    nullable = true,
    metadata = {}
  ),
  StructField(
    name = "price",
    dataType = IntegerType,
    nullable = true,
    metadata = {}
  ),
  StructField(
    name = "no_of_rooms",
    dataType = IntegerType,
    nullable = true,
    metadata = {}
  ),
  StructField(
    name = "open_fire",
    dataType = IntegerType,
    nullable = true,
    metadata = {}
  ),
  StructField(
    name = "terrace",
...

In [3]:
val haDF = spark.read.option("header","false").schema(schema).csv("house_apartment_cleaned_data.csv")
// haDF.show()
val houseNew = haDF.filter($"type_of_property" === "house" && $"state_of_building" === "as new")
// houseNew.show()
val totPriceNew = houseNew.groupBy("subtype_of_property").agg(sum("price").alias("Total_price_New"))
totPriceNew.show()

// For Renovated
val houseRenovated = haDF.filter($"type_of_property" === "house" && $"state_of_building" === "just renovated")
// houseRenovated.show()
val totPriceRen = houseRenovated.groupBy("subtype_of_property").agg(sum("price").alias("Total_price_Renovated"))
// totPriceRen.show()

25/07/25 09:40:03 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
25/07/25 09:40:03 INFO SharedState: Warehouse path is 'file:/C:/Users/dhsoni/spark-warehouse'.
25/07/25 09:40:06 INFO InMemoryFileIndex: It took 155 ms to list leaf files for 1 paths.
25/07/25 09:40:11 INFO FileSourceStrategy: Pushed Filters: IsNotNull(type_of_property),IsNotNull(state_of_building),EqualTo(type_of_property,house),EqualTo(state_of_building,as new)
25/07/25 09:40:11 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(type_of_property#1),isnotnull(state_of_building#9),(type_of_property#1 = house),(state_of_building#9 = as new)
25/07/25 09:40:14 INFO CodeGenerator: Code generated in 1421.3409 ms
25/07/25 09:40:14 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 198.6 KiB, free 2.1 GiB)
25/07/25 09:40:14 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 2.1 GiB)
25/07/25 0

+--------------------+---------------+
| subtype_of_property|Total_price_New|
+--------------------+---------------+
|         manor house|        5944000|
|               villa|      479928365|
|           farmhouse|        6437500|
|              castle|        1799999|
|  mixed use building|       42838711|
|     apartment block|       58532399|
|              chalet|        3953300|
|            bungalow|       12327000|
|      other property|        5257650|
|             mansion|       57171000|
|     country cottage|       26633900|
|               house|     1451322529|
|          town house|       38780409|
|exceptional property|       93186773|
+--------------------+---------------+



haDF: Dataset[Row] = [locality: int, type_of_property: string ... 9 more fields]
houseNew: Dataset[Row] = [locality: int, type_of_property: string ... 9 more fields]
totPriceNew: Dataset[Row] = [subtype_of_property: string, Total_price_New: bigint]
houseRenovated: Dataset[Row] = [locality: int, type_of_property: string ... 9 more fields]
totPriceRen: Dataset[Row] = [subtype_of_property: string, Total_price_Renovated: bigint]

In [4]:
val merge2 = (totPriceNew
             .join(totPriceRen, Seq("subtype_of_property"))
              .select(
                  col("subtype_of_property"),
                  col("Total_price_New"),
                  col("Total_price_Renovated")
              )
             )
// merge2.show()

val res = merge2.withColumn("Price Difference", $"Total_price_New" - $"Total_price_Renovated")
res.show()

25/07/25 09:40:25 INFO BlockManagerInfo: Removed broadcast_1_piece0 on 10.220.71.22:55817 in memory (size: 17.8 KiB, free: 2.1 GiB)
25/07/25 09:40:27 INFO FileSourceStrategy: Pushed Filters: IsNotNull(type_of_property),IsNotNull(state_of_building),EqualTo(type_of_property,house),EqualTo(state_of_building,as new),IsNotNull(subtype_of_property)
25/07/25 09:40:27 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(type_of_property#1),isnotnull(state_of_building#9),(type_of_property#1 = house),(state_of_building#9 = as new),isnotnull(subtype_of_property#2)
25/07/25 09:40:27 INFO FileSourceStrategy: Pushed Filters: IsNotNull(type_of_property),IsNotNull(state_of_building),EqualTo(type_of_property,house),EqualTo(state_of_building,just renovated),IsNotNull(subtype_of_property)
25/07/25 09:40:27 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(type_of_property#71),isnotnull(state_of_building#79),(type_of_property#71 = house),(state_of_building#79 = just renovated),isnotnull(subtype_of_pr

+--------------------+---------------+---------------------+----------------+
| subtype_of_property|Total_price_New|Total_price_Renovated|Price Difference|
+--------------------+---------------+---------------------+----------------+
|         manor house|        5944000|              2499000|         3445000|
|               villa|      479928365|             39234499|       440693866|
|           farmhouse|        6437500|              2629000|         3808500|
|  mixed use building|       42838711|             22061399|        20777312|
|     apartment block|       58532399|             25896399|        32636000|
|              chalet|        3953300|               284500|         3668800|
|            bungalow|       12327000|              3535500|         8791500|
|      other property|        5257650|              1328000|         3929650|
|     country cottage|       26633900|             10623000|        16010900|
|             mansion|       57171000|             15726900|    

merge2: Dataset[Row] = [subtype_of_property: string, Total_price_New: bigint ... 1 more field]
res: Dataset[Row] = [subtype_of_property: string, Total_price_New: bigint ... 2 more fields]

In [6]:
res.write.orc("Results/Res14.orc")
print("File Saved as orc")

25/07/25 09:42:43 INFO BlockManagerInfo: Removed broadcast_14_piece0 on 10.220.71.22:55817 in memory (size: 19.4 KiB, free: 2.1 GiB)


File Saved as orc